In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
user_polarity = pd.read_csv("USER_POLARITY_BARBERA.txt", delimiter = "\t", header=None)
user_polarity = user_polarity.drop([2], axis=1)
user_polarity = user_polarity.rename(columns={0: "user_ID", 1: "polarity_score"})
user_polarity.head(5)

,user_ID,polarity_score
0,1000010898,-1.375595
1,1000015790,1.817662
2,1000044026,1.817662
3,100004471,-0.798651
4,100004662,-1.618781


In [3]:
user_production_score = pd.read_csv("obamacare/users_production_scores.txt", 
                                    delimiter = "\t", header = None, lineterminator='\n')
user_production_score = user_production_score.rename(columns={0: "user_ID", 1: "production_score"})
user_production_score = user_production_score.drop([2,3], axis=1)
user_production_score.head(5)

,user_ID,production_score
0,334537201,0.734128
1,115094250,0.226682
2,2369916919,0.288123
3,1146470090,0.305678
4,95708744,0.347375


In [4]:
user_consumption_score = pd.read_csv("obamacare/users_consumption_scores.txt", 
                                     delimiter = "\t", header = None, lineterminator='\n')
user_consumption_score = user_consumption_score.rename(columns={0: "user_ID", 1: "consumption_score"})
user_consumption_score = user_consumption_score.drop([2,3], axis=1)
user_consumption_score.head(5)

,user_ID,consumption_score
0,334537201,0.738397
1,115094250,0.391875
2,2369916919,0.392721
3,1146470090,0.381466
4,95708744,0.347500


# data cleaning

In [8]:
def prepare_df(df):
    
    # the user has polarity score, production score, consumption score
    # and the tweet cites some domain
#     df = df[df[0].isin(user_polarity["user_ID"]) & \
#             df[0].isin(user_production_score["user_ID"]) & \
#             df[0].isin(user_consumption_score["user_ID"]) & \
#             df[1].isin(user_polarity["user_ID"]) & \
#             df[1].isin(user_production_score["user_ID"]) & \
#             df[1].isin(user_consumption_score["user_ID"])].dropna()

    df = df[df[0].isin(user_polarity["user_ID"]) & \
            df[0].isin(user_production_score["user_ID"]) & \
            df[0].isin(user_consumption_score["user_ID"])].dropna()
    
    df = df.rename(columns={
        0:  "user_ID",
        1:  "following_ID"})
    
    return df

In [9]:
conn = sqlite3.connect("obama_network2.db") 
df_iter = pd.read_csv("obamacare/FULL_FOLLOWER_NETWORK.txt.gz", 
                      delimiter = "\t", 
                      compression = "gzip",
                      header = None, 
                      lineterminator='\n',
                      error_bad_lines=False, 
                      chunksize = 100000,
                      names=list(range(2)))
for df in df_iter:
    df = prepare_df(df)
    if df is not None:
        df.to_sql("network", conn, if_exists = "append", index = False)

In [10]:
cursor = conn.cursor()

In [11]:
cursor.execute("SELECT COUNT(*) FROM network")
result = cursor.fetchone()
result

(36791137,)

In [12]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "network" (
"user_ID" INTEGER,
  "following_ID" INTEGER
)
